In [ ]:
import os
from functools import partial

import yaml
from IPython import display
import ipywidgets
from ipywidgets import Label, Dropdown, HBox, VBox, Text, Button, Valid
import traitlets
from traitlets import Unicode
import db

## File Upload Widget

In [ ]:
class FileWidget(ipywidgets.DOMWidget):
    _view_name = Unicode('FilePickerView').tag(sync=True)
    _view_module = Unicode('filepicker').tag(sync=True)

    value = Unicode().tag(sync=True)
    filename = Unicode().tag(sync=True)

    def __init__(self, **kwargs):
        """Constructor"""
        ipywidgets.DOMWidget.__init__(self, **kwargs) # Call the base.

        # Allow the user to register error callbacks with the following signatures:
        #    callback()
        #    callback(sender)
        self.errors = ipywidgets.CallbackDispatcher(accepted_nargs=[0, 1])

        # Listen for custom msgs
        self.on_msg(self._handle_custom_msg)

    def _handle_custom_msg(self, content):
        """Handle a msg from the front-end.

        Parameters
        ----------
        content: dict
            Content of the msg."""
        if 'event' in content and content['event'] == 'error':
            self.errors()
            self.errors(self)

In [ ]:
%%javascript

requirejs.undef('filepicker');

define('filepicker', ["jupyter-js-widgets"], function(widgets) {

    var FilePickerView = widgets.DOMWidgetView.extend({

        render: function() {
            // Render the view.
            this.setElement($('<input />')
                .attr('type', 'file'));
        },

        events: {
            // List of events and their handlers.
            'change': 'handle_file_change',
        },

        handle_file_change: function(evt) {
            // Handle when the user has changed the file.

            // Retrieve the first (and only!) File from the FileList object
            var file = evt.target.files[0];
            if (! file) { 
                // The file couldn't be opened.  Send an error msg to the
                // back-end.
                this.send({ 'event': 'error' });
                return ;
            }
            
            // check if has 'yml' extension

            // Read the file's textual content and set value to those contents.
            var that = this;
            var file_reader = new FileReader();
            file_reader.onload = function(e) {
                that.model.set('value', e.target.result);
                that.touch();
                console.log('sent from client side')
            }
            file_reader.readAsText(file);

            // Set the filename of the file.
            this.model.set('filename', file.name);
            this.touch();
        },
    });

    return {
        FilePickerView: FilePickerView
    };
});

---
## Utilities

In [ ]:
def get_exam_names():
    os.listdir()
    _, subdirs, _ = next(os.walk('.'))
    return [s[:-5] for s in subdirs if s.endswith('_data')]

def observe(widget, **kwargs):
    return partial(widget.observe, **kwargs)

---
## Widget Init

In [ ]:
yaml_upload_label = Label("Upload YAML", margin="0 10px 0 0")
yaml_upload = FileWidget()
yaml_upload_valid = Valid()
yaml_upload_status = Label()
yaml_upload_box = VBox([HBox([yaml_upload_label, yaml_upload, ]),
                        HBox([yaml_upload_status, yaml_upload_valid])])

yaml_upload_valid.layout.display = 'none'
yaml_upload_valid.layout.margin = "0 10px"

# Register an event to echo the filename when it has been changed.
@observe(yaml_upload, names="filename")
def file_loading(change):
    yaml_upload.value = ''
    yaml_upload_valid.layout.display = 'none'
    yaml_upload_status.value = "Loading " + yaml_upload.filename

# Register an event to echo the filename and contents when a file
# has been uploaded.
@observe(yaml_upload, names="value")
def file_loaded(change):
    if yaml_upload.value is None:
        return
    # validate yaml
    yaml_upload_status.value = "Validating " + yaml_upload.filename
    try:
        content = yaml.load(yaml_upload.value)
        if not isinstance(content, dict):
            raise yaml.error.YAMLError()
        test_name, _ = os.path.splitext(yaml_upload.filename)
        if content['name'] != test_name:
            raise ValueError('Test name does not match YAML filename')
    except ValueError as exc:
        yaml_upload_valid.value = False
        yaml_upload_valid.layout.display = 'inline'
        yaml_upload_status.value = str(exc)
        return
    except Exception:
        yaml_upload_valid.value = False
        yaml_upload_valid.layout.display = 'inline'
        yaml_upload_status.value = "Invalid YAML in " + yaml_upload.filename
        return


    # save yaml
    yaml_upload_status.value = "Saving " + yaml_upload.filename
    try:
        with open(yaml_upload.filename, 'w') as f:
            f.write(yaml_upload.value)
        os.makedirs(test_name + '_data', exist_ok=True)
    except Exception:
        yaml_upload_valid.value = False
        yaml_upload_status.value = "Failed to save " + yaml_upload.filename
    else:
        yaml_upload_valid.value = True
        yaml_upload_status.value = "Saved " + yaml_upload.filename
    finally:
        yaml_upload_valid.layout.display = 'inline'
        # XXX: use of global variables
        # update list of exams
        select_exam.options = [None] + get_exam_names()

# Register an event to print an error message when a file could not
# be opened.  Since the error messages are not handled through
# traitlets but instead handled through custom msgs, the registration
# of the handler is different than the two examples above.  Instead
# the API provided by the CallbackDispatcher must be used.
def file_failed():
    yaml_upload_valid.value = False
    yaml_upload_valid.layout.display = 'inline'
    yaml_upload_status.value = 'Could not load file contents of ' + yaml_upload.filename
yaml_upload.errors.register_callback(file_failed)

In [ ]:
pdf_label = Label("Exam PDF URL", margin="7px 10px")
pdf_url = Text()
pdf_import = Button(description="Import exam data")
pdf_import_box = HBox([pdf_label ,pdf_url, pdf_import])
pdf_import_box.layout.display = 'none'

In [ ]:
select_exam = Dropdown(description='', options= [None] + get_exam_names())

@observe(select_exam, names="value")
def _(state):
    display = {False: 'none', True: 'inline'}
    value = state['new']
    yaml_upload_box.layout.display = display[value is None]
    pdf_import_box.layout.display = display[value is not None]

---
## UI Layout

In [ ]:
select_exam

In [ ]:
VBox([yaml_upload_box, pdf_import_box])